# © Artur Czarnecki. All rights reserved.
# Integrax framework

# Notebook 11 — ChatGPT-like E2E (behavior-level)

This notebook is an **integration / behavior** test that exercises the Drop-In Knowledge Runtime end-to-end, in a way that resembles a real ChatGPT usage pattern.

## What we test (behavior)
- Multi-session behavior (A/B/C) with **isolated session history** and **shared user LTM**
- User LTM persistence + recall across sessions
- Session-level consolidation (history → summary) without cross-session leakage
- RAG ingestion + Q&A over a document
- Websearch as a context layer that affects the final answer
- Tools execution (tool + LLM) without breaking the **user-last invariant**
- Reasoning enabled for observability, but **not persisted into user-visible history**

## What we do NOT test (intentionally out of scope for this notebook)
- Retry / fallback logic for empty LLM outputs
- Formal “adapter contract” beyond `generate_messages(...) -> str`
- Tools contract v1 (final answer vs context-only)
- Prompt tuning, quality scoring, reranking
- Debug refactors (e.g., removing getattr) unless strictly required to run the notebook

## Hard invariants
- **User-last invariant:** the last message sent to the core LLM must always be `role="user"`
- No empty assistant answers in the final output
- No memory leakage between sessions (history isolation)

## Requirements
This notebook expects your local environment to provide:
- Intergrax sources on PYTHONPATH
- LLM credentials (e.g., `OPENAI_API_KEY`) if using OpenAI adapters
- Vectorstore backend deps (Chroma/Qdrant) if enabling RAG/LTM vector retrieval


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [5]:
import os, sys
from datetime import datetime
from pathlib import Path
import uuid
from intergrax.llm_adapters.base import LLMAdapter, LLMAdapterRegistry, LLMProvider
from intergrax.runtime.drop_in_knowledge_mode.config import ReasoningConfig, RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager
from intergrax.memory.user_profile_manager import UserProfileManager
from intergrax.memory.stores.in_memory_user_profile_store import InMemoryUserProfileStore
from intergrax.rag.embedding_manager import EmbeddingManager
from intergrax.rag.vectorstore_manager import VSConfig, VectorstoreManager

# =====================================================================
# Global test identifiers / paths (no tests executed in this cell)
# =====================================================================

USER_ID = "user_chatgpt_like_001"

SESSION_A = "sess_chatgpt_like_A"
SESSION_B = "sess_chatgpt_like_B"
SESSION_C = "sess_chatgpt_like_C"

RUN_ID = datetime.utcnow().strftime("%Y%m%d_%H%M%S") + "_" + uuid.uuid4().hex[:8]

BASE_DIR = Path(os.getcwd()).resolve()
ARTIFACTS_DIR = BASE_DIR / "_artifacts" / "notebook_11_chatgpt_like" / RUN_ID
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

# Separate vectorstore collections (1 instance = 1 collection_name)
# - RAG_DOCS: document ingestion/retrieval
# - USER_LTM: user long-term memory retrieval
RAG_DIR = ARTIFACTS_DIR / "vs_rag_docs"
LTM_DIR = ARTIFACTS_DIR / "vs_user_ltm"
RAG_DIR.mkdir(parents=True, exist_ok=True)
LTM_DIR.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------------------
# LLM adapter (real adapter, no wrappers)
# - assumes env is configured (OPENAI_API_KEY etc.)
# ---------------------------------------------------------------------
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)


# ---------------------------------------------------------------------
# Embeddings + vectorstore (real managers)
# Pick providers you actually use in your repo/env.
# ---------------------------------------------------------------------

embed_manager = EmbeddingManager(
    provider="ollama",
)

rag_vs = VectorstoreManager(
    config=VSConfig(
        provider="chroma",
        collection_name=f"rag_docs_{RUN_ID}",
        chroma_persist_directory=str(RAG_DIR),
    )
)

# User LTM vectorstore
ltm_vs = VectorstoreManager(
    VSConfig(
        provider="chroma",
        collection_name=f"user_ltm_{RUN_ID}",
        chroma_persist_directory=str(LTM_DIR),
    )
)

# ---------------------------------------------------------------------
# Stores
# ---------------------------------------------------------------------
session_store = InMemorySessionStorage()
user_profile_store = InMemoryUserProfileStore()

# ---------------------------------------------------------------------
# Managers
# ---------------------------------------------------------------------
user_profile_manager = UserProfileManager(
    store=user_profile_store,
    embedding_manager=embed_manager,
    vectorstore_manager=ltm_vs,
)

session_manager = SessionManager(
    storage=session_store,
    user_profile_manager=user_profile_manager,
)

# ---------------------------------------------------------------------
# Runtime config
# ---------------------------------------------------------------------
config = RuntimeConfig(
    llm_adapter=llm_adapter,
    embedding_manager=embed_manager,
    vectorstore_manager=rag_vs,
    enable_user_profile_memory=True,
    enable_org_profile_memory=False,
    enable_user_longterm_memory=True,
    enable_rag=True,
    enable_websearch=True,
    tools_mode="off",
    reasoning_config=ReasoningConfig()
)


# =====================================================================
# Runtime factory (keeps Cell-2..Cell-8 focused on behavior)
# =====================================================================
def build_runtime(*, override_config: dict | None = None, **runtime_kwargs) -> DropInKnowledgeRuntime:
    """
    Build a DropInKnowledgeRuntime instance.
    - override_config: dict of RuntimeConfig fields to override (shallow).
    - runtime_kwargs: runtime init kwargs (e.g., ingestion_service, context_builder, prompt builders).
    """
    if override_config:
        for k, v in override_config.items():
            setattr(config, k, v)

    return DropInKnowledgeRuntime(
        config=config,
        session_manager=session_manager,
        ingestion_service=runtime_kwargs.get("ingestion_service"),
        context_builder=runtime_kwargs.get("context_builder"),
        rag_prompt_builder=runtime_kwargs.get("rag_prompt_builder"),
        websearch_prompt_builder=runtime_kwargs.get("websearch_prompt_builder"),
        history_prompt_builder=runtime_kwargs.get("history_prompt_builder"),
    )


print("BOOTSTRAP OK")
print("RUN_ID:", RUN_ID)
print("ARTIFACTS_DIR:", str(ARTIFACTS_DIR))
print("USER_ID:", USER_ID)
print("SESSION_A:", SESSION_A)
print("SESSION_B:", SESSION_B)
print("SESSION_C:", SESSION_C)


C:\Users\XPS\AppData\Local\Temp\ipykernel_22996\4077648888.py:25: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  RUN_ID = datetime.utcnow().strftime("%Y%m%d_%H%M%S") + "_" + uuid.uuid4().hex[:8]


[intergraxVectorstoreManager] Initialized provider=chroma, collection=rag_docs_20251222_082640_1a1d5e42
[intergraxVectorstoreManager] Existing count: 0
[intergraxVectorstoreManager] Initialized provider=chroma, collection=user_ltm_20251222_082640_1a1d5e42
[intergraxVectorstoreManager] Existing count: 0
BOOTSTRAP OK
RUN_ID: 20251222_082640_1a1d5e42
ARTIFACTS_DIR: D:\Projekty\intergrax\notebooks\drop_in_knowledge_mode\_artifacts\notebook_11_chatgpt_like\20251222_082640_1a1d5e42
USER_ID: user_chatgpt_like_001
SESSION_A: sess_chatgpt_like_A
SESSION_B: sess_chatgpt_like_B
SESSION_C: sess_chatgpt_like_C
